![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2F03+-+BigQuery+ML+%28BQML%29&file=BQML+Remote+Model+on+Vertex+AI+Endpoint.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/03%20-%20BigQuery%20ML%20%28BQML%29/BQML%20Remote%20Model%20on%20Vertex%20AI%20Endpoint.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2F03%2520-%2520BigQuery%2520ML%2520%2528BQML%2529%2FBQML%2520Remote%2520Model%2520on%2520Vertex%2520AI%2520Endpoint.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/03%20-%20BigQuery%20ML%20%28BQML%29/BQML%20Remote%20Model%20on%20Vertex%20AI%20Endpoint.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/03%20-%20BigQuery%20ML%20%28BQML%29/BQML%20Remote%20Model%20on%20Vertex%20AI%20Endpoint.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# BQML Predictions With Remote Model (Vertex AI)

This notebooks show how to register a remote model, on Vertex AI Prediction Endpoints, as a BigQuery ML model.  This makes it possible to do inference/prediction within BigQuery using the [ML.PREDICT()](https://cloud.google.com/bigquery/docs/reference/standard-sql/bigqueryml-syntax-predict) function.  This can be very helpful in cases where the model is already hosted for online prediction and might even needing serving infrastructure like GPUs or might be too large for [importing into BigQuery ML](https://cloud.google.com/bigquery/docs/reference/standard-sql/inference-overview#inference_using_imported_models).

$$\textrm{BigQuery ML} \Longrightarrow \textrm{Vertex AI Prediction Endpoint}$$


**Prerequisites**

This notebook requires an existing model hosted on a Vertex AI Prediction Endpoint to server predictions. It makes uses of any of the prior notebooks in [this series](./readme.md) which each export a BigQuery ML model to Vertex AI Model Registry and host them for online prediction on a Vertex AI Prediction Endpoint.  

**References**
- Tutorial [Make predictions with remote models on Vertex AI](https://cloud.google.com/bigquery/docs/bigquery-ml-remote-model-tutorial)
    - This features a another tutorial within this repository: [BQML Remote Model Tutorial](./BQML%20Remote%20Model%20Tutorial.md)

---
## Colab Setup

To run this notebook in Colab click [![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/03%20-%20BigQuery%20ML%20(BQML)/BQML%20Remote%20Model%20on%20Vertex%20AI%20Endpoint.ipynb) and run the cells in this section.  Otherwise, skip this section.

This cell will authenticate to GCP (follow prompts in the popup).

In [1]:
PROJECT_ID = 'statmike-mlops-349915' # replace with project ID

In [473]:
try:
    import google.colab
    from google.colab import auth
    auth.authenticate_user()
    !gcloud config set project {PROJECT_ID}
except Exception:
    pass

---
## Setup

installations

This notebook uses the Python Clients for
- Google Service Usage
    - to enable APIs (BigQuery Connection)

In [3]:
try:
    import google.cloud.service_usage_v1
except ImportError:
    !pip install google-cloud-service-usage -q

inputs:

In [4]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [5]:
REGION = 'us-central1'
EXPERIMENT = 'remote-model'
SERIES = 'bqml'

# source data
BQ_PROJECT = PROJECT_ID
BQ_DATASET = 'fraud'
BQ_TABLE = 'fraud_prepped'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters

packages:

In [7]:
from google.cloud import bigquery
from google.cloud import aiplatform
from google.cloud import service_usage_v1

import json

clients:

In [8]:
bq = bigquery.Client(project = PROJECT_ID)
aiplatform.init(project = PROJECT_ID, location = REGION)
su_client = service_usage_v1.ServiceUsageClient()

### Enable APIs

BigQuery Remote models use the BigQuery Connection API. 

Options for enabeling:  In this notebook option 2 is used.
 1. Use the APIs & Services page in the console: https://console.cloud.google.com/apis
     - `+ Enable APIs and Services`
     - Search for BigQuery Connection
 2. Use [Google Service Usage](https://cloud.google.com/service-usage/docs) API from Python
     - [Python Client For Service Usage](https://github.com/googleapis/python-service-usage)
     - [Python Client Library Documentation](https://cloud.google.com/python/docs/reference/serviceusage/latest)
     
The following code cells use the Service Usage Client to:
- get the state of the service
- if 'DISABLED':
    - Try enabling the service and return the state after trying
- if 'ENABLED' print the state for confirmation

#### BigQuery Connection

In [9]:
bigqueryconnection = su_client.get_service(
    request = service_usage_v1.GetServiceRequest(
        name = f'projects/{PROJECT_ID}/services/bigqueryconnection.googleapis.com'
    )
).state.name


if bigqueryconnection == 'DISABLED':
    print(f'API is currently {bigqueryconnection} for project: {PROJECT_ID}')
    print(f'Trying to Enable...')
    operation = su_client.enable_service(
        request = service_usage_v1.EnableServiceRequest(
            name = f'projects/{PROJECT_ID}/services/bigqueryconnection.googleapis.com'
        )
    )
    response = operation.result()
    if response.service.state.name == 'ENABLED':
        print(f'API is now enabled for project: {PROJECT_ID}')
    else:
        print(response)
else:
    print(f'API already enabled for project: {PROJECT_ID}')

API already enabled for project: statmike-mlops-349915


---
## Get Data For Predictions

### Retrieve Records For Prediction

In [13]:
n = 10
pred = bq.query(
    query = f"""
        SELECT * EXCEPT({VAR_TARGET}, {VAR_OMIT}, splits)
        FROM {BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}
        WHERE splits='TEST'
        LIMIT {n}
        """
).to_dataframe()

In [14]:
pred.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
0,35337,1.092844,-0.013230,1.359829,2.731537,-0.707357,0.873837,-0.796130,0.437707,0.396770,...,-0.240428,0.037603,0.380026,-0.167647,0.027557,0.592115,0.219695,0.036970,0.010984,0.0
1,60481,1.238973,0.035226,0.063003,0.641406,-0.260893,-0.580097,0.049938,-0.034733,0.405932,...,-0.265080,-0.060003,-0.053585,-0.057718,0.104983,0.537987,0.589563,-0.046207,-0.006212,0.0
2,139587,1.870539,0.211079,0.224457,3.889486,-0.380177,0.249799,-0.577133,0.179189,-0.120462,...,-0.374356,0.196006,0.656552,0.180776,-0.060226,-0.228979,0.080827,0.009868,-0.036997,0.0
3,162908,-3.368339,-1.980442,0.153645,-0.159795,3.847169,-3.516873,-1.209398,-0.292122,0.760543,...,-0.923275,-0.545992,-0.252324,-1.171627,0.214333,-0.159652,-0.060883,1.294977,0.120503,0.0
4,165236,2.180149,0.218732,-2.637726,0.348776,1.063546,-1.249197,0.942021,-0.547652,-0.087823,...,-0.250653,0.234502,0.825237,-0.176957,0.563779,0.730183,0.707494,-0.131066,-0.090428,0.0


Shape as instances: dictionaries of key:value pairs for only features used in model

In [15]:
newobs = pred.to_dict(orient='records')
#newobs[0]

In [16]:
len(newobs)

10

In [17]:
newobs[0]

{'Time': 35337,
 'V1': 1.0928441854981998,
 'V2': -0.0132303486713432,
 'V3': 1.35982868199426,
 'V4': 2.7315370965921004,
 'V5': -0.707357349219652,
 'V6': 0.8738370029866129,
 'V7': -0.7961301510622031,
 'V8': 0.437706509544851,
 'V9': 0.39676985012996396,
 'V10': 0.587438102569443,
 'V11': -0.14979756231827498,
 'V12': 0.29514781622888103,
 'V13': -1.30382621882143,
 'V14': -0.31782283120234495,
 'V15': -2.03673231037199,
 'V16': 0.376090905274179,
 'V17': -0.30040350116459497,
 'V18': 0.433799615590844,
 'V19': -0.145082264348681,
 'V20': -0.240427548108996,
 'V21': 0.0376030733329398,
 'V22': 0.38002620963091405,
 'V23': -0.16764742731151097,
 'V24': 0.0275573495476881,
 'V25': 0.59211469704354,
 'V26': 0.219695164116351,
 'V27': 0.0369695108704894,
 'V28': 0.010984441006191,
 'Amount': 0.0}

---

## Get A Model For Predictions

This section retrieves a currently active model being used on a Vertex AI Prediction Endpoint.

>If you already know the location of your model files in a GCS Bucket then this section can be bypassed by storing the model location with: `model_uri = 'gs://bucket/path/to/files'.

In [18]:
# Series 04 creates scikit-learn based models
PREVIOUS_SERIES = '03'

### Get Endpoint

Reference: [aiplatform.Endpoint](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Endpoint)

In [19]:
endpoints = aiplatform.Endpoint.list(filter = f"labels.series={PREVIOUS_SERIES}")
endpoint = endpoints[0]

In [20]:
print(f'Review the Endpoint in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/endpoints/{endpoint.name}?project={PROJECT_ID}')

Review the Endpoint in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/endpoints/4867252103141130240?project=statmike-mlops-349915


In [21]:
[list(newobs[0].values())]

[[35337,
  1.0928441854981998,
  -0.0132303486713432,
  1.35982868199426,
  2.7315370965921004,
  -0.707357349219652,
  0.8738370029866129,
  -0.7961301510622031,
  0.437706509544851,
  0.39676985012996396,
  0.587438102569443,
  -0.14979756231827498,
  0.29514781622888103,
  -1.30382621882143,
  -0.31782283120234495,
  -2.03673231037199,
  0.376090905274179,
  -0.30040350116459497,
  0.433799615590844,
  -0.145082264348681,
  -0.240427548108996,
  0.0376030733329398,
  0.38002620963091405,
  -0.16764742731151097,
  0.0275573495476881,
  0.59211469704354,
  0.219695164116351,
  0.0369695108704894,
  0.010984441006191,
  0.0]]

In [23]:
endpoint.predict(instances = newobs[0:1]).predictions

[{'latent_col_5': 0.291676432,
  'mean_squared_log_error': 0.00712372037,
  'mean_absolute_error': 0.164848313,
  'latent_col_4': -0.232606351,
  'mean_squared_error': 0.0966151,
  'latent_col_7': 0.716156483,
  'latent_col_1': -0.505942464,
  'latent_col_2': -0.287162453,
  'latent_col_3': -0.78166455,
  'latent_col_8': 0.572722137,
  'latent_col_6': -0.392200977}]

### Review The Model Information

Reference: [aiplatform.Model](https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.Model)

In [24]:
vertex_model = aiplatform.Model(
    model_name = endpoint.list_models()[0].model + f'@{endpoint.list_models()[0].model_version_id}'
)

In [25]:
vertex_model.display_name

'03_autoencoder'

In [26]:
vertex_model.version_id

'2'

In [27]:
vertex_model.name

'model_03_autoencoder'

In [28]:
vertex_model.uri

'gs://statmike-mlops-349915/03/autoencoder/models/20230321130321/model'

In [29]:
!gsutil ls {vertex_model.uri}

gs://statmike-mlops-349915/03/autoencoder/models/20230321130321/model/checkpoint
gs://statmike-mlops-349915/03/autoencoder/models/20230321130321/model/graph.pbtxt
gs://statmike-mlops-349915/03/autoencoder/models/20230321130321/model/model.ckpt-91300.data-00000-of-00001
gs://statmike-mlops-349915/03/autoencoder/models/20230321130321/model/model.ckpt-91300.index
gs://statmike-mlops-349915/03/autoencoder/models/20230321130321/model/model.ckpt-91300.meta
gs://statmike-mlops-349915/03/autoencoder/models/20230321130321/model/saved_model.pb
gs://statmike-mlops-349915/03/autoencoder/models/20230321130321/model/variables/


In [31]:
print(f'Review the model in the Vertex AI Model Registry:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{vertex_model.name}/versions/{vertex_model.version_id}/properties?project={PROJECT_ID}')

Review the model in the Vertex AI Model Registry:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/model_03_autoencoder/versions/2/properties?project=statmike-mlops-349915


---
## BQML Remote Model

Creating a BigQuery ML Remote Model has two components: a [BigQuery Cloud Resource Connection](https://cloud.google.com/bigquery/docs/create-cloud-resource-connection) and a BigQuery Remote Model that uses the connection.

### Create A BigQuery Cloud Resource Connection

In [32]:
!bq version

This is BigQuery CLI 2.0.86


In [38]:
!bq ls --connection --location={REGION} --project_id={PROJECT_ID}

No connections found.


In [39]:
# create BigQuery Cloud Resource Connection
!bq mk --connection --location={REGION} --project_id={PROJECT_ID} --connection_type=CLOUD_RESOURCE {SERIES}_{EXPERIMENT}

Connection 1026793852137.us-central1.bqml_remote-model successfully created


In [41]:
bqml_connection = !bq show --format prettyjson --connection {PROJECT_ID}.{REGION}.{SERIES}_{EXPERIMENT}
bqml_connection = json.loads(''.join(bqml_connection))
service_account = bqml_connection['cloudResource']['serviceAccountId']
service_account

'bqcx-1026793852137-zfly@gcp-sa-bigquery-condel.iam.gserviceaccount.com'

Assign the service account [Vertex AI user role](https://cloud.google.com/vertex-ai/docs/general/access-control#aiplatform.user):

Alternatively, Do This in the Console:
- Console > IAM > + Grand Access:
    - New Principal = <enter value of serviceAccountId above>
    - Role = Vertex AI User
    - Click Save

In [42]:
# assign vertex ai user role to the service account of the BigQuery Cloud Resource Connection
!gcloud projects add-iam-policy-binding {PROJECT_ID} --member=serviceAccount:{service_account} --role=roles/aiplatform.user

Updated IAM policy for project [statmike-mlops-349915].
bindings:
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform-cc.iam.gserviceaccount.com
  role: roles/aiplatform.customCodeServiceAgent
- members:
  - serviceAccount:service-1026793852137@gcp-sa-aiplatform.iam.gserviceaccount.com
  role: roles/aiplatform.serviceAgent
- members:
  - serviceAccount:bqcx-1026793852137-zfly@gcp-sa-bigquery-condel.iam.gserviceaccount.com
  role: roles/aiplatform.user
- members:
  - serviceAccount:service-1026793852137@gcp-sa-artifactregistry.iam.gserviceaccount.com
  role: roles/artifactregistry.serviceAgent
- members:
  - serviceAccount:1026793852137-compute@developer.gserviceaccount.com
  role: roles/bigquery.admin
- members:
  - serviceAccount:1026793852137@cloudservices.gserviceaccount.com
  role: roles/bigquery.dataOwner
- members:
  - serviceAccount:1026793852137@cloudbuild.gserviceaccount.com
  role: roles/cloudbuild.builds.builder
- members:
  - serviceAccount:service-1026793

### Get The Model Inputs and Outputs

To see the input and output specification for the TensorFlow model use the [SavedModel CLI](https://www.tensorflow.org/guide/saved_model#details_of_the_savedmodel_command_line_interface):

```
MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:
signature_def['serving_default']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['image_bytes'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: encoded_image_string_tensor:0
    inputs['key'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: key:0
```

In [80]:
model_graph = !saved_model_cli show --dir {vertex_model.uri} --all

In [82]:
model_graph = [x.strip() for x in model_graph if x.strip()[0:5] in ['input', 'outpu', 'dtype', 'shape', 'name:']]

In [122]:
# parse this into a dictionary:
inputs = ''
outputs = ''
format_mapper = dict(DT_DOUBLE = 'FLOAT64', DT_FLOAT = 'FLOAT64', DT_INT = 'INT64', DT_STRING = 'STRING')
for x in range(int(len(model_graph)/4)):
    variable = model_graph[x*4:x*4+4]
    #variable[0] = 'feature:' + variable[0].split('[')[1].split(']')[0]
    if variable[0].split('[')[0] == 'inputs':
        inputs += variable[0].split("['")[1].split("']")[0]
        inputs += ' '
        shape = tuple(int(x) for x in variable[2].split('(')[1].split(')')[0].split(','))
        if len(shape) > 1:
            inputs += f"ARRAY<{format_mapper[variable[1].split(' ')[-1]]}>" + ', '
        else:
            inputs += f"{format_mapper[variable[1].split(' ')[-1]]}" + ', '
    else:
        outputs += variable[0].split("['")[1].split("']")[0]
        outputs += ' '
        shape = tuple(int(x) for x in variable[2].split('(')[1].split(')')[0].split(','))
        if len(shape) > 1:
            outputs += f"ARRAY<{format_mapper[variable[1].split(' ')[-1]]}>" + ', '
        else:
            outputs += f"{format_mapper[variable[1].split(' ')[-1]]}" + ', '
    #print(variable)

#print(inputs)
#print(outputs)
    


In [123]:
print(inputs)

Amount FLOAT64, Time FLOAT64, V1 FLOAT64, V10 FLOAT64, V11 FLOAT64, V12 FLOAT64, V13 FLOAT64, V14 FLOAT64, V15 FLOAT64, V16 FLOAT64, V17 FLOAT64, V18 FLOAT64, V19 FLOAT64, V2 FLOAT64, V20 FLOAT64, V21 FLOAT64, V22 FLOAT64, V23 FLOAT64, V24 FLOAT64, V25 FLOAT64, V26 FLOAT64, V27 FLOAT64, V28 FLOAT64, V3 FLOAT64, V4 FLOAT64, V5 FLOAT64, V6 FLOAT64, V7 FLOAT64, V8 FLOAT64, V9 FLOAT64, 


In [124]:
print(outputs)

latent_col_1 FLOAT64, latent_col_2 FLOAT64, latent_col_3 FLOAT64, latent_col_4 FLOAT64, latent_col_5 FLOAT64, latent_col_6 FLOAT64, latent_col_7 FLOAT64, latent_col_8 FLOAT64, mean_absolute_error FLOAT64, mean_squared_error FLOAT64, mean_squared_log_error FLOAT64, 


### Create Remote Model

In [125]:
query = f"""
CREATE OR REPLACE MODEL `{BQ_PROJECT}.{BQ_DATASET}.{SERIES}_{EXPERIMENT}`
    INPUT ({inputs})
    OUTPUT ({outputs})
    REMOTE WITH CONNECTION `{PROJECT_ID}.{REGION}.{SERIES}_{EXPERIMENT}`
    OPTIONS(
        endpoint = 'https://{REGION}-aiplatform.googleapis.com/v1/{endpoint.resource_name}'
    )
"""
job = bq.query(query = query)
job.result()
job.state

'DONE'

### Predict with ML.PREDICT

Get predictions from the remote model within BigQuery using the `ML.PREDICT` function.  This sends records from the query statment to the remote model for serving prediction back to BigQuery as a single function call.

In [128]:
query = f"""
SELECT *
FROM ML.PREDICT (MODEL `{BQ_PROJECT}.{BQ_DATASET}.{SERIES}_{EXPERIMENT}`,(
    SELECT * 
    FROM `{BQ_PROJECT}.{BQ_DATASET}.{BQ_TABLE}`
    WHERE splits = 'TEST'
    LIMIT 1
    )
  )
"""
pred = bq.query(query = query).to_dataframe()

In [129]:
pred

,latent_col_1,latent_col_2,latent_col_3,latent_col_4,latent_col_5,latent_col_6,latent_col_7,latent_col_8,mean_absolute_error,mean_squared_error,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,-0.505942,-0.287162,-0.781665,-0.232606,0.291676,-0.392201,0.716156,0.572722,0.164848,0.096615,...,-0.167647,0.027557,0.592115,0.219695,0.03697,0.010984,0.0,0,a1b10547-d270-48c0-b902-7a0f735dadc7,TEST


In [130]:
endpoint.predict(instances = newobs[0:1]).predictions

[{'latent_col_7': 0.716156483,
  'mean_absolute_error': 0.164848313,
  'mean_squared_error': 0.0966151,
  'mean_squared_log_error': 0.00712372037,
  'latent_col_2': -0.287162453,
  'latent_col_8': 0.572722137,
  'latent_col_6': -0.392200977,
  'latent_col_4': -0.232606351,
  'latent_col_1': -0.505942464,
  'latent_col_3': -0.78166455,
  'latent_col_5': 0.291676432}]

---
## Clean Up

In [37]:
# remove BigQuery Cloud Resource Connection
#!bq rm --connection {PROJECT_ID}.{REGION}.{SERIES}_{EXPERIMENT}